# Salary Prediction
This project predicts salary based on job descriptions.

## Features:
1. jobId
2. companyId
3. jobType
4. degree
5. major
6. industry
7. yearsExperience
8. milesFromMetropolis
9. salary

## Import libraries
This project uses machine learning approach, so lots of sklearn classes are used.

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# set display to 3 decimal max
pd.set_option('precision', 3)

## Load data
First thing first, unzip it and see what's inside!

In [ ]:
zipfile_path = os.path.join("SalaryPredictions.zip")
zipfile_ref = zipfile.ZipFile(zipfile_path, 'r')
zipfile_ref.extractall()
zipfile_ref.close()

print(os.listdir("data"))

In [ ]:
test_df = pd.read_csv(os.path.join("data", "test_features.csv"), header=0, index_col=0)
train_df = pd.read_csv(os.path.join("data", "train_features.csv"), header=0, index_col=0)
train_df['salary'] = pd.read_csv(os.path.join("data", "train_salaries.csv"), header=0, index_col=0)

In [ ]:
print(train_df.info())
display(train_df.head(10))

In [ ]:
print(test_df.info())
display(train_df.head(10))

## Clean data
Before we dive in to data analysis, we should perform basic cleaning. Look for:
1. Any missing values (np.None or np.NaN) in entire training and testing set
    * from train_df.info() and test_df.info(), there's no null cell
2. Any duplicated samples in training set
3. Any invalid yearsExperience (yearsExperience <= 0) in training set
4. Any invalid milesFromMetropolis (milesFromMetropolis < 0) in training set
5. Any invalid salary (salary < 0) in training set

In [ ]:
print("Any duplicated rows in training set?", train_df.duplicated().sum())
print("Any invalid yearsExperience in training set?", train_df.yearsExperience.lt(0).sum())
print("Any invalid milesFromMetropolis in training set?", train_df.milesFromMetropolis.lt(0).sum())
print("Any invalid salary in training set?", train_df.salary.le(0).sum())

Need to:
1. Drop duplicated jobs in training set
2. Drop jobs with salary <= 0. 

Since we have 1,000,000 samples, we have enough data to handle removing those jobs.

In [ ]:
# Drop duplicated jobs in training set
train_df.drop_duplicates(inplace=True)

# Drop jobs with <=0 salary
train_df = train_df.drop(train_df.index[train_df.salary.le(0)])

train_df_copy = train_df.copy()
print("Training set now has shape:", train_df.shape)

## Explore data
Now that we've done the basic cleaning, let's look at each feature one by one and generate some statistics reports to further analyze our data.
### Company ID
In real-life, if a company is well-known or an international organization, the salary is likely to be higher than local startups. We can generate a statistics report to find out which company pays more in general.

In [ ]:
train_df.groupby(['companyId']).salary.describe().sort_values('mean', ascending=False)

Since there are 63 companies, let's randomly select some companies and see their statistics:

In [ ]:
companies = train_df.companyId.unique()
pick_n_companies = 15
rand_companies = companies[np.random.randint(1, len(companies), pick_n_companies)]

for i in range(pick_n_companies):
    if i==0:
        groups = train_df[['companyId', 'salary']].loc[train_df.companyId == rand_companies[i], :]
    else:
        group = train_df[['companyId', 'salary']].loc[train_df.companyId == rand_companies[i], :]
        groups = pd.concat([groups, group], axis=0)

groups.boxplot(by='companyId')
plt.show()

Observations on both statistics reports:
1. Similar distribution
2. Average salary varies only a little
3. Similar deviation
4. The min, 25%, 50%, and 75% are almost the same between companies

The salary distributions are very similar that no company particularly stands out and gives higher or lower wage, and of course no trend or pattern in this feature that relates to salary. Unless there are other features depending on companyId, so far it's not giving me useful info and it's likely to be dropped.

### Job Types
There are 8 types of job: CEO, CFO, CTO, JANITOR, JUNIOR, MANAGER, SENIOR and VICE_PRESIDENT.

It's reasonable to assume the chief positions offers much higher than JANITOR and JUNIOR.

In [ ]:
print(train_df.groupby(['jobType']).salary.describe().sort_values(by='mean', ascending=False))
train_df[['jobType', 'salary']].boxplot(by='jobType')

Looks like there's a rule to follow:
1. Each job type has 124,000~126,000 sample
2. The differences in average salary between each job type is the largest among all featuers
    * Average salary ranges from 145 to 70
3. Both CFO and CTO offer same salary and distribution
4. The ranking of average salary is: CEO > CFO = CTO > VP > MANAGER > SENIOR > JUNIOR >> JANITOR
5. The deviation also decreases with average salary
6. Most positions are slightly positive skewed Gaussian distribution

Some data transformation is needed to emphasize the rules in jobType vs. salary. 

To visualize more on the skewness, use histograms and calculation of skewness:

In [ ]:
jobTypes = train_df.jobType.unique()
plt.figure(figsize = (14, 6))

for i in range(len(jobTypes)):
    plt.subplot(3, 3, i+1)
    plt.hist(train_df.salary[train_df.jobType == jobTypes[i]])
    plt.title(jobTypes[i])
plt.show()

train_df.groupby(['jobType']).salary.skew()

The histogram and skew table back up our previous assumption of positive skewness. As shown in table, the skewness are all positive and very small, and the histogram for each position only slightly lean toward left. 

The skewness level is: CEO < CFO = CTO < VICE_PRESIDENT < MANAGER = SENIOR < JUNIOR < JANITOR, which is opposite of average salary ranking! My guessing would be that lower positions usually have more room to negotiate for more salary due to other factors, such as world class company might offer more for the same position than local startups, or people with prior experience in the industry tends to be offered more than university graduates, and so on. And people interviewing for higher position might not be for the money, so the positions tends to offer standard salary.

### Degree
There are 5 types of degree: BACHLORS, DOCTORAL, HIGH_SCHOOL, MASTERS, and NONE

I expect that more professional position asks for higher educatation, so jobs requiring DOCTORAL degree might offer much higher than HIGH SCHOOL graduated. But will it be the same for DOCTORAL vs. MASTER degree? How much of a gap in salary will it be?

In [ ]:
train_df.groupby(['degree']).size()

From number of jobs, we see that:
1. Number of jobs asking for NONE degree is almost the same as asking for HIGH_SCHOOL degree.
2. Number of jobs asking for BACHELORS, MASTERS, and DOCTORAL are almost the same.

In [ ]:
print(train_df.groupby(['degree']).describe().salary.sort_values(by='mean', ascending=False))
train_df[['degree', 'salary']].boxplot(by='degree')

From statistics reports, it can be observed that:
1. Jobs asking for HIGH_SCHOOL degree offer almost the same salary as not asking any degree (NONE). 
2. Jobs asking for HIGH_SCHOOL degree or NONE offer much less than BACHELORS and above. 
3. Distribution for BACHELORS, MASTERS, and DOCTORAL shifts and narrows in same rate.
4. The ranking of average salary is: DOCTORAL > MASTERS > BACHELORS >> HIGH_SCHOOL > NONE.
5. Except HIGH_SCHOOL, the ranking of deviation is the same as average salary
    * HIGH_SCHOOL has highest deviation
    * Maybe some jobs are intern positions that allow students to continue higher education while working, thus the salary is negotiable judging from their potential
    * Maybe other jobs are general labor jobs that don't require any professional knowledge, thus paid less
6. All degree are Gaussian distributed. And maybe positively skewed.

Combine with finding from number of jobs, seems like data can be seperated to two groups:
1. NONE and HIGH_SCHOOL: more job opening but lower offer
2. BACHELORS, MASTERS, DOCTORAL: less job openings and higher offer

Will it simplify the degree feature too much? Need to experiments them to find out!

Again, let's verify the Gaussian distribution and skewness with histograms.

In [ ]:
jobTypes = train_df.degree.unique()
plt.figure(figsize = (14, 6))

for i in range(len(jobTypes)):
    plt.subplot(2, 3, i+1)
    plt.hist(train_df.salary[train_df.degree == jobTypes[i]])
    plt.title(jobTypes[i])
plt.show()

train_df.groupby(['degree']).salary.skew()

As expected, it's Gaussian and positive skewed. 

The level of skewness is: MASTERS = NONE = BACHELORS > DOCTORAL >> HIGH_SCHOOL, all with very little difference. It's interesting that HIGH_SCHOOL degree is skewed the least and also offered second to least, suggesting that jobs requiring HIGH_SCHOOL are more fixed compare to other degrees.

### Major
There are 9 majors: BIOLOGY, BUSINESS, CHEMISTRY, COMPSCI (computer science), ENGINEERING, LITERATURE, MATH, NONE, and PHYSICS

I expect the technical (ENGINEERING and COMPSCI) and BUSINESS major might be offered more than pure science (BIOLOGY, CHEMISTRY, MATH, PHYSICS) and LITERATURE major since they're more practical (eg. designing product, analyzing market trend, etc.) and less theoratical or economic (eg. research papers, studying Shakespeare, etc.).

In [ ]:
train_df.groupby(['major']).size()

Number of jobs asking for no major is more than other jobs combined and also more than half of the data. The imbalance data says that more than half of the offerings don't look at applicant's major, everyone can apply. But this raises some questions:
* Does it reflect on salary? 
* If the major is irrelevent, do those jobs have lower wage because it's not professional? 
     * For example, workers at warehouse or clark at supermarket might not need be any major to work.
* Does it suggest us that the major doesn't provide valueable information since so many jobs don't require any and we should focus on other features such as yearsExperience? 
* Is it related to other features?
    * For example, OLD and SERVICE industry are willing to hire more NONE major employees than WEB or FINANCE?
    * For example, is employee likely to be paid more with higher education diploma in any major?

Let's confirm it from statistics report.

In [ ]:
print(train_df.groupby(['major']).salary.describe().sort_values(by='mean', ascending=False))
train_df[['major', 'salary']].boxplot(by='major')

1. NONE obviously has lower distribution than others by great amount.
2. The ranking of average salary is: ENGINEERING > BUSINESS > MATH > COMPSCI > PHYSICS > CHEMISTRY > BIOLOGY > LITERATURE >> NONE
3. BUSINESS and ENGINEERING have larger deviation
4. LITERATURE has smallest deviation
5. Ranking of deviation is similar to mean, except NONE has higher deviation
    * Lower paid jobs usually offers the standard salary
    * Higher paid jobs depends more on other factors, thus have wider range of salary
    * There can be various jobs that don't require any major, thus it also heavily depends on other features
6. Gaussian distributed and skew to positive

This verifies our initial guessing that jobs with no major requirement might be less professional and thus paid less. But consider the imbalanced data, it's too early to assume all jobs not requiring major pay less since we're not sure if there are other factors influencing the salary. Need more digging!

In [ ]:
jobTypes = train_df.major.unique()
plt.figure(figsize = (14, 6))

for i in range(len(jobTypes)):
    plt.subplot(3, 3, i+1)
    plt.hist(train_df.salary[train_df.major == jobTypes[i]])
    plt.title(jobTypes[i])
plt.show()

train_df.groupby(['major']).salary.skew()

BUSINESS is the most skewed and BIOLOGY and NONE are least skewed. But compare to other features like jobType, all majors are skewed at similar rate.

### Industry
There are 7 types of industry: AUTO (automobile), EDUCATION, FINANCE, HEALTH, OIL, SERVICE, and WEB

I expect FINANCE and WEB pays relatively higher and SERVICE pays lower.

In [ ]:
train_df.groupby(['industry']).size()

Each industry has around 142,800 openings.

In [ ]:
print(train_df.groupby(['industry']).salary.describe().sort_values(by='mean', ascending=False))
train_df[['industry', 'salary']].boxplot(by='industry')

Findings:
1. FINANCE and OIL(!) have highest wage and most stretched deviation
2. FINANCE and OIL have the same distribution
3. The ranking of average salary is: FINANCE = OIL > WEB > HEALTH > AUTO > SERVICE > EDUCATION
4. The ranking of deviation is the same as average salary
    * There's a significant drop between WEB and HEALTH
    * Is there any relationship between group:(OIL, FINANCE, WEB) and group:(HEALTH, AUTO, SERVICE, EDUCATION)?
5. The differences between each industry's average salary are the smallest among all features
    * The average salary ranges from 130 to 99
5. Gaussian distribution and positive skew

Let's look at the salary histograms and skewness

In [ ]:
jobTypes = train_df.industry.unique()
plt.figure(figsize = (14, 6))

for i in range(len(jobTypes)):
    plt.subplot(3, 3, i+1)
    plt.hist(train_df.salary[train_df.industry == jobTypes[i]])
    plt.title(jobTypes[i])
plt.show()

train_df.groupby(['industry']).salary.skew()

The skewness is the relatively small among all features and skews at similar rate. Similar to the deviations, there's a gap in skewness between group:(OIL, WEB, FINANCE) and group:(AUTO, HEALTH, SERVICE, EDUCATION).

### Years of Experience
Span from 0 to 24 years. Let's select a few and see their distributions.

I expect the more years of experience a job asks, the more it pays. But is there a plateau? Is the relationship linear?

In [ ]:
np.unique(train_df.yearsExperience)

In [ ]:
years= np.arange(24+1)
for i in range(len(years)):
    if i==0:
        groups = train_df[['yearsExperience', 'salary']][train_df.yearsExperience == years[i]]
    else:
        group = train_df[['yearsExperience', 'salary']][train_df.yearsExperience == years[i]]
        groups = pd.concat([groups, group], axis=0)

groups.boxplot(by='yearsExperience')
plt.show()

As expected:
1. The yearsExperience and average salary have linear relationship
2. The deviation increases with average salary
3. Gaussian distribution and positive skew

In [ ]:
years= np.arange(0, 24+1, 3)
plt.figure(figsize = (14, 6))

for i in range(len(years)):
    plt.subplot(3, 3, i+1)
    plt.hist(train_df.salary[train_df.yearsExperience == years[i]])
    plt.title(years[i])
plt.show()

train_df.groupby(['yearsExperience']).salary.skew()

The skewness is the smallest of all features and skews at similar rate. Overall, the skewness increases with yearsExperience.

And to show the linear relationship, here's the plot of yearsExperience vs. average salary.

In [ ]:
print(train_df.groupby(['yearsExperience']).salary.describe())
train_df.groupby(['yearsExperience']).salary.describe()['mean'].plot(x='yearsExperience', y='salary')

Now that's an obvious slope of 2 and offset 90 linear relationship.

yearsExperience is an independent feature, Gaussian distributed, and has a simple linear trend pattern. If we scale it down using standard scaler, we won't distored the pattern and it'll release some burdent in training.

### Miles from metropolis
Span from 0 to 99 miles. Like yearsExperience, let's select a few and see their distributions.

I expect companies having offices in metropolis are likely to have higher net worth, thus the closer to metropolis, the more it pays. Again, is there a plateau? Is the relationship linear?

In [ ]:
np.unique(train_df.milesFromMetropolis)

In [ ]:
miles= np.arange(0, 99+1, 4)
for i in range(len(miles)):
    if i==0:
        groups = train_df[['milesFromMetropolis', 'salary']][train_df.milesFromMetropolis == miles[i]]
    else:
        group = train_df[['milesFromMetropolis', 'salary']][train_df.milesFromMetropolis == miles[i]]
        groups = pd.concat([groups, group], axis=0)

groups.boxplot(by='milesFromMetropolis')
plt.show()

1. It's a inverse linear relationship
2. Deviation decreases with average salary
3. Gaussian distribution and positive skew

In [ ]:
miles= np.arange(0, 99+1, 12)
plt.figure(figsize = (14, 6))

for i in range(len(miles)):
    plt.subplot(3, 3, i+1)
    plt.hist(train_df.salary[train_df.milesFromMetropolis == miles[i]])
    plt.title(miles[i])
plt.show()

train_df.groupby(['milesFromMetropolis']).salary.skew()[miles]

The skewness is the small among all features and skews at similar rate. There's no obvious relationship between skewness and and milesFromMetropolis.

And to show the linearity, here's the plot of milesFromMetropolis vs. average salary.

In [ ]:
print(train_df.milesFromMetropolis.describe())
train_df.groupby(['milesFromMetropolis']).salary.describe()['mean'].plot(x='milesFromMetropolis', y='salary')

It's an obvious slope of -0.4 and offset 135 linear trend.

Similar to yearsExperience, milesFromMetropolis is an independent feature, Gaussian distributed, and has a simple (reversed) linear trend pattern.

### Data analysis conclusion
1. Without further data transformation, all features have obvious pattern/ranking
2. All features are Gaussian distributed and positive skewness
3. Higher average salary often pairs with wider distribution (larger deviation)
4. **companyId** provides little to no information related to salary
5. **jobType** has highest variance
    * Avearage salary ranges from 145 to 70
6. **degree** can potentially be split to two groups
    * NONE and HIGH_SCHOOL: more job opening but lower offer
    * BACHELORS, MASTERS, DOCTORAL: less job openings and higher offer
7. **major** has imbalanced data
    * More than half of dataset don't require a major
8. **industry** has smallest variance
    * Average salary ranges from 130 to 99
9. **yearsExperience** and **milesFromMetropolis** both are independent and linearly related to salary

## Transform data
The goal is to transform our data according to certain rules to further reveal the hidden pattern.

Obviously, we have to transform strings to meaningful numerical values. companyId, jobType, degree, major, and industry are the columns in dataset that need convertion. We have three options:
1. One-Hot Encoder
2. Label Encoder
3. Custome Label Encoder

Let's try all three and see which method gives me features that are:
1. Independent
2. Most correlated to salary

### Outliers

### Encoders
To find the best encoding method, let's use 50,000 samples for testing and see which yields best correlated features.
#### One Hot Encoder
In this case, one-hot encoding 50,000 samples takes too long, so use 500 samples instead. Also show first 40 out of 95 correlations with salary.

### Encoders
To find the best encoding method, let's use 50,000 samples for testing and see which yields best correlated features.
#### One Hot Encoder
In this case, one-hot encoding 50,000 samples takes too long, so use 500 samples instead. Also show first 40 out of 95 correlations with salary.

In [ ]:
# TODO: write a class instead: class OneHotEncodingFeatures:
def one_hot_encoder(train, features):
    for feature in features:
        categories = train[feature].unique()
        for category in categories:
            train[feature + "_" + category] = (train[feature] == category).astype(int)
        train = train.drop(columns=feature)
    return train

In [ ]:
# because one-hot encoding 50,000 samples takes too long, use 5,000 samples instead
example_set = train_df[:5000].copy()
display(one_hot_encoder(example_set, ['jobType', 'degree', 'major', 'industry', 'companyId']).head(5))

show_corr = 40
one_hot_encoder(example_set, ['jobType', 'degree', 'major', 'industry', 'companyId']).corr().salary.iloc[:show_corr]

One-Hot encoding is the standard way to transform strings to numericals, but this gives too many features (95 columns) and takes too long to generate. And if we look into the correlation matrix, the maximum negative correlation is jobType_JANITOR with magnitude of 0.457 and positive correlation is 0.368 from yearsExperience, which didn't need to be one-hot encoded. Thus, one-hot encoding doesn't give me strong correlated features.

#### Label Encoder
This method uses sklearn.preprocessing.LabelEncoder() to assign numerical values starting from 0.

In [ ]:
# TODO: write a class instead: class LabelEncodingFeatures:
def label_encoder(train, features, test=None):
    encoder = sklearn.preprocessing.LabelEncoder()
    for feature in features:
        train[feature] = encoder.fit_transform(train[feature])
        if test:
            test[feature] = encoder.transform(test[feature])

    return train, test

In [ ]:
example_set = train_df[:50000].copy()
features = ['jobType', 'degree', 'major', 'industry', 'companyId']

display(label_encoder(example_set, features).head(5))

correlation = label_encoder(example_set, features).corr()
print(correlation['salary'])

plt.figure(figsize=(15, 10))
sns.heatmap(correlation, annot=True, vmin=-1, cmap="RdBu_r")

With Label Encoder:
1. No new features
2. Except companyId and industry, other features have good correlation with salary
3. Except major and degree, other pairs of features are independent from each other

This approach looks promissing! Let's see if we can lift the perfomance up a notch with custom Laber Encoder.

#### Custom Label Encoder
This method uses the ranking of average salary to assign numerical values starting from 1.
* Starts from 1 because rank() starts from 1, and I don't want to "deactivate" any label of each feature with 0
* Average salaries are rounded down, so labels with same average salary will have the same rank
* companyId: 63 companies are ranked according to average salary
* **jobType**: CEO = 1, CFO = 2, CTO = 2, VP = 4, MANAGER = 5, SENIOR = 6, JUNIOR = 7, JANITOR = 8
* **degree**: DOCTORAL = 1, MASTERS = 2, BACHELORS = 3, HIGH_SCHOOL = 4, NONE = 5
* **major**: ENGINEERING = 1, BUSINESS = 2, MATH = 3, COMPSCI = 4, PHYSICS = 5, CHEMISTRY = 6, BIOLOGY = 7, LITERATURE = 8, NONE = 9
* **industry**: FINANCE = 1, OIL = 1, WEB = 3, HEALTH = 4, AUTO = 5, SERVICE = 6, EDUCATION = 7

In [ ]:
# TODO: write a class instead: class RankFeatures:
def mean_salary_encoder(train, features, test=None):
    for feature in features:
        salary_dict = dict(train.groupby([feature]).salary.mean())
        train[feature] = train[feature].map(salary_dict)
        if test:
            test[feature] = test[feature].map(salary_dict)

    return train, test

In [ ]:
example_set = train_df[:50000].copy()
features = ['jobType', 'degree', 'major', 'industry', 'companyId']

display(mean_salary_encoder(example_set, features)[0].head(5))

correlation = mean_salary_encoder(example_set, features)[0].corr()
print(correlation.salary)

plt.figure(figsize=(15, 10))
sns.heatmap(correlation, annot=True, vmin=-1, cmap="RdBu_r")

From correlation matrix, we observe:
1. Correlation matrix of custom Label Encoder looks like correlation matrix of Label Encoder but better
2. Between Salary and other features, only companyId has low correlation with salary
    * Consider removing companyId
3. Between feature to feature, Major and Industry are highly correlated.
    * Consider merging Major and Industry to avoid possible noise
4. Between two features, pairs other than Major-Industry have low correlation, meaning two features are relatively independent from each other

Thus makes it the plausible encoder for our case.

### Transform data and test performance
Now we've selected our encoder, next is to transform data. From the correlation matrix with custom label encoder, we see some potential options for feature selection:
1. Drop companyId
2. Merge major or degree

We'll perform a quick test with LinearRegression model to see which of the following data transformaiton method yields the least MSE:
1. Keep companyId and not merge degree and major
2. Drop companyId and not merge degree and major
3. Keep companyId and merge degree and major
4. Drop companyId and merge degree and major

In [ ]:
def group_k_columns(train, features, col_name="group", test=None):
    group = train.groupby(features).salary
    Q1 = group.quantile(0.25)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR

    salary_dictionaries = []
#     salary_dictionaries.append(dict(lower_bound,    name = col_names + "group_lower"))
    salary_dictionaries.append(dict(group.min(),    name = col_name + "group_min"))
    salary_dictionaries.append(dict(Q1,             name = col_name + "group_Q1"))
    salary_dictionaries.append(dict(group.mean(),   name = col_name + "group_mean"))
    salary_dictionaries.append(dict(group.median(), name = col_name + "group_median"))
    salary_dictionaries.append(dict(Q3,             name = col_name + "group_Q3"))
#     salary_dictionaries.append(dict(upper_bound,    name = col_names + "group_upper"))
    salary_dictionaries.append(dict(group.max(),    name = col_names + "group_max"))
    
    for dictionary in salary_dictionaries:
        salary = []
        for index in train.index:
            combination = []
            for i in range(len(features)):
                combination.append(train.loc[index, features[i]])
            salary.append(dictionary[tuple(combination)])
        train[dictionary['name']] = pd.Series(salary, index=train.index)

        if test:
            salary = []
            for index in test.index:
                combination = []
                for i in range(len(features)):
                    combination.append(test.loc[index, features[i]])
                salary.append(dictionary[tuple(combination)])
            test[dictionary['name']] = pd.Series(salary, index=test.index)
        
    return train, test

In [ ]:
def feature_engineer(train, features, col_name, test=None):
    train, test = group_k_columns(train, features, col_name, test)
    train, test = mean_salary_encoder(train, features, test)

    return train, test

In [ ]:
features = ['companyId', 'jobType', 'degree', 'major', 'industry']
stat_col_name = 'CJDMI'
correlation = feature_engineer(train_df.deepcopy(), features, col_name)[0].corr()

print(correlation.salary)

plt.figure(figsize=(15, 10))
sns.heatmap(correlation, annot=True, vmin=-1, cmap="RdBu_r")

## Create models
### Baseline model
Create a naive model and measure its efficiency as my performance metrics baseline. I'm using average salary for each industry as my baseline model.

In [ ]:
industry_dict = dict(train_df.groupby(['industry']).describe().salary.mean())
baseline_true = train_df.salary.values.astype(float)
baseline_pred = train.industry.map(industry_dict)

mse = mean_squared_error(baseline_true, baseline_pred)
mae = mean_absolute_error(baseline_true, baseline_pred)
print("Baseline: MSE=%.3f\tMAE=%.3f" % (mse, mae))

### Regression and ensamble models

In [ ]:
features = ['companyId', 'jobType', 'degree', 'major', 'industry']
stat_col_name = 'CJDMI'
train = feature_engineer(train_df.deepcopy(), features, col_name)[0]

model_list = [('LR', LinearRegression()), 
              ('LASSO', Lasso()), 
              ('EN', ElasticNet()), 
              ('CART', DecisionTreeRegressor()),
              ("RF", RandomForestRegressor()),
              ## Boosting
              ('AB', AdaBoostRegressor()), 
              ('GBR', GradientBoostingRegressor()), 
              ## Bagging
              ('RF', RandomForestRegressor(n_estimators=60, max_depth=15, min_samples_split=80, max_features=8)), 
              ('ET', ExtraTreesRegressor())]

for name, model in model_list:
    scores = sklearn.model_selection.cross_val_score(model, train.drop(columns='salary'), train.salary, scoring='neg_mean_squared_error')
    mean_mse.append(-1.0 * np.mean(scores))
    print("%s:\tMSE=%.3f" % (name, -1.0 * np.mean(scores)))

Pick models that have lower MSE: LinearRegression, Lasso, ElasticNet, and GradientBoostingRegressor. Tune hyperparameters and try other adjustments to improve performance.

### Scalng

In [ ]:
train = feature_engineer(train_df.deepcopy(), ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')[0]

model_list = [("scaledLR",    Pipeline([("Scaler", StandardScaler()), ("LR", LinearRegression())])),
              ("scaledLASSO", Pipeline([("Scaler", StandardScaler()), ("LASSO", Lasso())])),
              ("scaledEN",    Pipeline([("Scaler", StandardScaler()), ("EN", ElasticNet())])),
              ("scaledGBR",   Pipeline([("Scaler", StandardScaler()), ("GBR", GradientBoostingRegressor(max_depth=6, loss='huber'))]))]

for name, model in model_list:
    scores = sklearn.model_selection.cross_val_score(model, train.drop(columns='salary'), train.salary, scoring='neg_mean_squared_error')
    print("%s:\tMSE=%.3f" % (name, -1.0 * np.mean(scores)))

Standardize the data doesn't improve the performance. Let's try tuning hyperparameters.

### Tuning
#### Gradient Boosting Regressor

In [ ]:
train = feature_engineer(train_df.deepcopy(), ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')[0]

for n_est in [50, 100, 200]:
    for loss in ['ls', 'huber']:
        for depth in [6, 8, 10]:
            model = GradientBoostingRegressor(n_estimators=100, loss='ls', max_depth=8)
            scores = sklearn.model_selection.cross_val_score(model, train.drop(columns='salary'), train.salary, scoring='neg_mean_squared_error')
            mse = -1.0 * np.mean(scores)
            print("(%d, %s, %.3f):\tMSE=%.3f" % (n_est, 'ls', depth, mse))

            model.fit(train.drop(columns='salary'), train.salary)
            feature_importance = model.feature_importances_
            for i in range(len(feature_importance)):
                print("%s  %.6f" % (train.drop(columns='salary').columns[i], feature_importance[i]))

## Deploy solution

In [ ]:
train = feature_engineer(train_df.deepcopy(), ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')[0]

model = GradientBoostingRegressor(n_estimators=100, loss='ls', max_depth=8)
scores = sklearn.model_selection.cross_val_score(model, train.drop(columns='salary'), train.salary, scoring='neg_mean_squared_error')
mse = -1.0 * np.mean(scores)

model.fit(train.drop(columns='salary'), train.salary)
feature_importance = model.feature_importances_
for i in range(len(feature_importance)):
    print("%s  %.6f" % (train.drop(columns='salary').columns[i], feature_importance[i]))

predictions = model.predict(test_features).round(3)
test_salary = pd.DataFrame(predictions, index=test_features.index)
test_salary.to_csv(os.path.join("data", "test_salary_prediction.csv"))

In [ ]:
train = group_k_columns(train, 2, ['degree', 'major'], "DM")
train = group_k_columns(train, 2, ['jobType', 'degree'], "JD")
train = group_k_columns(train, 2, ['jobType', 'major'], "JM")
train = group_k_columns(train, 3, ['jobType', 'degree', 'major'], "JDM")
train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
# train = group_k_columns(train, 7, ['companyId', 'jobType', 'degree', 'major', 'industry', 'yearsExperience', 'milesFromMetropolis'], 'CJDMIYM')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry', 'yearsExperience', 'milesFromMetropolis'])
    
(50, ls, 6.000):	MSE=334.113
(50, ls, 8.000):	MSE=328.809
(50, ls, 10.000):	MSE=327.740
(50, huber, 6.000):	MSE=334.282
(50, huber, 8.000):	MSE=328.878
    

    
salary_mean_dict = dict(train.groupby([feature]).salary.mean())
# salary_mean_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())
    
train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])
(50, ls, 12.000):	MSE=332.343
(150, ls, 8.000):	MSE=327.946
    
companyId  0.000700
jobType  0.005244
degree  0.003090
major  0.001900
industry  0.003331
yearsExperience  0.155236
milesFromMetropolis  0.107955
CJDMI_mean  0.707699
CJDMI_median  0.014845



# salary_mean_dict = dict(train.groupby([feature]).salary.mean())
salary_mean_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())
train = group_k_columns(train, 3, ['degree', 'major', 'industry'], "DMI")
train = group_k_columns(train, 4, ['companyId', 'degree', 'major', 'industry'], "CDMI")
train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])

(150, ls, 8.000):	MSE=327.145
companyId  0.000053
jobType  0.006356
degree  0.001235
major  0.000139
industry  0.000722
yearsExperience  0.155082
milesFromMetropolis  0.107684
DMI_mean  0.004945
DMI_median  0.001223
CDMI_mean  0.001024
CDMI_median  0.001079
CJDMI_mean  0.706410
CJDMI_median  0.014051



# salary_dict = dict(train.groupby([feature]).salary.mean())
salary_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())

train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])

salary_mean_dict = dict(train.groupby(col_names).salary.mean())
salary_mean_dict['name'] = new_col_name_mean
salary_Q0_dict = dict(train.groupby(col_names).salary.quantile(0))
salary_Q0_dict['name'] = new_col_name_Q0
salary_Q1_dict = dict(train.groupby(col_names).salary.quantile(0.25))
salary_Q1_dict['name'] = new_col_name_Q1
salary_Q2_dict = dict(train.groupby(col_names).salary.median())
salary_Q2_dict['name'] = new_col_name_Q2
salary_Q3_dict = dict(train.groupby(col_names).salary.quantile(0.75))
salary_Q3_dict['name'] = new_col_name_Q3
salary_Q4_dict = dict(train.groupby(col_names).salary.quantile(1))
salary_Q4_dict['name'] = new_col_name_Q4
salary_dictionaries = [salary_mean_dict, salary_Q0_dict, salary_Q1_dict, salary_Q2_dict, salary_Q3_dict, salary_Q4_dict]

(100, ls, 8.000):	MSE=306.777
companyId  0.000021
jobType  0.003827
degree  0.002999
major  0.001062
industry  0.002066
yearsExperience  0.152040
milesFromMetropolis  0.104977
CJDMI_mean  0.658974
CJDMI_Q0  0.009351
CJDMI_Q1  0.031602
CJDMI_Q2  0.006909
CJDMI_Q3  0.008939
CJDMI_Q4  0.017233



salary_dict = dict(train.groupby([feature]).salary.mean())
# salary_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())

train = group_k_columns(train, 3, ['degree', 'major', 'industry'], 'DMI')
train = mean_salary_encoder(train, ['jobType'])
train = train.drop(columns=['companyId', 'degree', 'major', 'industry'])

(100, ls, 8.000):	MSE=356.407
jobType  0.415251
degree  0.000361
major  0.000179
industry  0.002465
yearsExperience  0.186674
milesFromMetropolis  0.129748
DMI_mean  0.130898
DMI_Q0  0.002157
DMI_Q1  0.008558
DMI_Q2  0.022342
DMI_Q3  0.087371
DMI_Q4  0.013997



salary_mean_dict = dict(train.groupby(col_names).salary.mean())
salary_mean_dict['name'] = new_col_name_mean
# salary_Q0_dict = dict(train.groupby(col_names).salary.quantile(0))
# salary_Q0_dict['name'] = new_col_name_Q0
# salary_Q1_dict = dict(train.groupby(col_names).salary.quantile(0.25))
# salary_Q1_dict['name'] = new_col_name_Q1
# salary_Q2_dict = dict(train.groupby(col_names).salary.median())
# salary_Q2_dict['name'] = new_col_name_Q2
# salary_Q3_dict = dict(train.groupby(col_names).salary.quantile(0.75))
# salary_Q3_dict['name'] = new_col_name_Q3
# salary_Q4_dict = dict(train.groupby(col_names).salary.quantile(1))
# salary_Q4_dict['name'] = new_col_name_Q4

(100, ls, 8.000):	MSE=356.385
jobType  0.414720
degree  0.000792
major  0.000316
industry  0.007666
yearsExperience  0.186760
milesFromMetropolis  0.129649
DM_mean  0.260096



# salary_dict = dict(train.groupby([feature]).salary.mean())
salary_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())

(100, ls, 8.000):	MSE=356.357
jobType  0.414795
degree  0.000755
major  0.000276
industry  0.007868
yearsExperience  0.186807
milesFromMetropolis  0.129701
DM_mean  0.259799



train = group_k_columns(train, 3, ['jobType', 'yearsExperience', 'milesFromMetropolis'], 'JYM')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])
# train = train.drop(columns=['companyId', 'degree', 'major'])

(100, ls, 8.000):	MSE=354.008
companyId  0.001198
jobType  0.003756
degree  0.054679
major  0.041514
industry  0.111630
yearsExperience  0.000877
milesFromMetropolis  0.001105
JYM_mean  0.785242



salary_dict = dict(train.groupby([feature]).salary.mean())
# salary_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())

# train = group_k_columns(train, 3, ['jobType', 'yearsExperience', 'milesFromMetropolis'], 'JYM')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])
# train = train.drop(columns=['companyId', 'degree', 'major'])

(100, ls, 8.000):	MSE=357.078
companyId  0.001164
jobType  0.415428
degree  0.111364
major  0.038076
industry  0.117292
yearsExperience  0.186888
milesFromMetropolis  0.129788



salary_mean_dict = dict(train.groupby(col_names).salary.mean())
salary_mean_dict['name'] = new_col_name_mean
salary_lower_dict = dict(lower_bound)
salary_lower_dict['name'] = new_col_name_Q0
salary_Q1_dict = dict(Q1)
salary_Q1_dict['name'] = new_col_name_Q1
salary_Q2_dict = dict(train.groupby(col_names).salary.median())
salary_Q2_dict['name'] = new_col_name_Q2
salary_Q3_dict = dict(Q3)
salary_Q3_dict['name'] = new_col_name_Q3
salary_upper_dict = dict(upper_bound)
salary_upper_dict['name'] = new_col_name_Q4
salary_dictionaries = [salary_mean_dict, salary_lower_dict, salary_Q1_dict, salary_Q2_dict, salary_Q3_dict, salary_upper_dict]

train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])

(40, ls, 8.000):	MSE=321.275
companyId  0.000038
jobType  0.004394
degree  0.001057
major  0.001053
industry  0.002115
yearsExperience  0.153585
milesFromMetropolis  0.105777
CJDMI_mean  0.676035
CJDMI_lower  0.002281
CJDMI_Q1  0.034198
CJDMI_median  0.007061
CJDMI_Q3  0.010344
CJDMI_upper  0.002063

(100, ls, 8.000):	MSE=318.217
companyId  0.000241
jobType  0.004977
degree  0.001390
major  0.001454
industry  0.002724
yearsExperience  0.153228
milesFromMetropolis  0.106019
CJDMI_mean  0.671498
CJDMI_lower  0.002739
CJDMI_Q1  0.034679
CJDMI_median  0.007618
CJDMI_Q3  0.010905
CJDMI_upper  0.002526

(150, ls, 8.000):	MSE=318.538
companyId  0.000416
jobType  0.005046
degree  0.001426
major  0.001551
industry  0.002871
yearsExperience  0.153244
milesFromMetropolis  0.106380
CJDMI_mean  0.669683
CJDMI_lower  0.002943
CJDMI_Q1  0.034792
CJDMI_median  0.007855
CJDMI_Q3  0.011059
CJDMI_upper  0.002733

(200, ls, 8.000):	MSE=319.019
companyId  0.000576
jobType  0.005077
degree  0.001444
major  0.001626
industry  0.002970
yearsExperience  0.153258
milesFromMetropolis  0.106739
CJDMI_mean  0.668141
CJDMI_lower  0.003097
CJDMI_Q1  0.034884
CJDMI_median  0.008074
CJDMI_Q3  0.011209
CJDMI_upper  0.002904



train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
# train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])
train = train.drop(columns=['companyId', 'jobType', 'degree', 'major', 'industry'])

(100, ls, 8.000):	MSE=323.537
yearsExperience  0.153815
milesFromMetropolis  0.106334
CJDMI_mean  0.680445
CJDMI_lower  0.002935
CJDMI_Q1  0.035180
CJDMI_median  0.008045
CJDMI_Q3  0.010879
CJDMI_upper  0.002365



salary_mean_dict = dict(train.groupby(col_names).salary.mean())
salary_mean_dict['name'] = new_col_name_mean
salary_min_dict = dict(train.groupby(col_names).salary.min())
salary_min_dict['name'] = new_col_name_min
salary_lower_dict = dict(lower_bound)
salary_lower_dict['name'] = new_col_name_Q0
salary_Q1_dict = dict(Q1)
salary_Q1_dict['name'] = new_col_name_Q1
salary_Q2_dict = dict(train.groupby(col_names).salary.median())
salary_Q2_dict['name'] = new_col_name_Q2
salary_Q3_dict = dict(Q3)
salary_Q3_dict['name'] = new_col_name_Q3
salary_upper_dict = dict(upper_bound)
salary_upper_dict['name'] = new_col_name_Q4
salary_max_dict = dict(train.groupby(col_names).salary.max())
salary_max_dict['name'] = new_col_name_max
salary_dictionaries = [salary_mean_dict, salary_min_dict, salary_lower_dict, salary_Q1_dict, 
                       salary_Q2_dict, salary_Q3_dict, salary_upper_dict, salary_max_dict]

(100, ls, 8.000):	MSE=313.483
yearsExperience  0.152787
milesFromMetropolis  0.105388
CJDMI_mean  0.665288
CJDMI_min  0.010648
CJDMI_lower  0.001203
CJDMI_Q1  0.031204
CJDMI_median  0.007509
CJDMI_Q3  0.007989
CJDMI_upper  0.001416
CJDMI_max  0.016567



train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])
train = train.drop(columns=['companyId', 'jobType', 'degree', 'major', 'industry'])

(100, ls, 8.000):	MSE=313.491
yearsExperience  0.152785
milesFromMetropolis  0.105390
CJDMI_mean  0.665287
CJDMI_min  0.010648
CJDMI_lower  0.001202
CJDMI_Q1  0.031206
CJDMI_median  0.007507
CJDMI_Q3  0.007989
CJDMI_upper  0.001419
CJDMI_max  0.016567



train = group_k_columns(train, 5, ['companyId', 'jobType', 'degree', 'major', 'industry'], 'CJDMI')
train = mean_salary_encoder(train, ['companyId', 'jobType', 'degree', 'major', 'industry'])
# train = train.drop(columns=['companyId', 'jobType', 'degree', 'major', 'industry'])

(100, ls, 8.000):	MSE=307.004
companyId  0.000191
jobType  0.003687
degree  0.002976
major  0.001054
industry  0.002109
yearsExperience  0.151975
milesFromMetropolis  0.104943
CJDMI_mean  0.658703
CJDMI_min  0.008978
CJDMI_lower  0.001002
CJDMI_Q1  0.030981
CJDMI_median  0.007029
CJDMI_Q3  0.007857
CJDMI_upper  0.001499
CJDMI_max  0.017016



salary_min_dict = dict(train.groupby(col_names).salary.min())
salary_min_dict['name'] = new_col_name_min
salary_Q1_dict = dict(Q1)
salary_Q1_dict['name'] = new_col_name_Q1
salary_mean_dict = dict(train.groupby(col_names).salary.mean())
salary_mean_dict['name'] = new_col_name_mean
salary_Q2_dict = dict(train.groupby(col_names).salary.median())
salary_Q2_dict['name'] = new_col_name_Q2
salary_Q3_dict = dict(Q3)
salary_Q3_dict['name'] = new_col_name_Q3
salary_upper_dict = dict(upper_bound)
salary_upper_dict['name'] = new_col_name_Q4
salary_max_dict = dict(train.groupby(col_names).salary.max())
salary_max_dict['name'] = new_col_name_max
salary_dictionaries = [salary_mean_dict, salary_min_dict, """salary_lower_dict""", salary_Q1_dict, 
                       salary_Q2_dict, salary_Q3_dict, salary_upper_dict, salary_max_dict]

(100, ls, 8.000):	MSE=306.779
companyId  0.000199
jobType  0.003814
degree  0.003028
major  0.001074
industry  0.002091
yearsExperience  0.152010
milesFromMetropolis  0.104899
CJDMI_mean  0.658765
CJDMI_min  0.009157
CJDMI_Q1  0.031207
CJDMI_median  0.006925
CJDMI_Q3  0.007889
CJDMI_upper  0.001902
CJDMI_max  0.017041



# salary_dict = dict(train.groupby([feature]).salary.mean())
# salary_dict = dict(np.floor(train.groupby([feature]).salary.mean()).rank())
salary_dict = {}
for cat in train[feature].unique():
    salary_dict[cat] = train[train[feature]==cat].salary.mode()[0]

(100, ls, 8.000):	MSE=307.692
companyId  0.000168
jobType  0.003588
degree  0.002864
major  0.001076
industry  0.001801
yearsExperience  0.152146
milesFromMetropolis  0.105048
CJDMI_mean  0.659155
CJDMI_min  0.009254
CJDMI_Q1  0.031239
CJDMI_median  0.006950
CJDMI_Q3  0.007850
CJDMI_upper  0.001870
CJDMI_max  0.016991



salary_dictionaries = [salary_mean_dict, salary_min_dict, salary_Q1_dict, #salary_lower_dict
                       salary_Q2_dict, salary_Q3_dict, salary_max_dict]#salary_upper_dict,

(100, ls, 8.000):	MSE=307.603
companyId  0.000186
jobType  0.003569
degree  0.002906
major  0.001058
industry  0.001777
yearsExperience  0.152123
milesFromMetropolis  0.105045
CJDMI_mean  0.659153
CJDMI_min  0.009398
CJDMI_Q1  0.031698
CJDMI_median  0.006880
CJDMI_Q3  0.008998
CJDMI_max  0.017209

### Ensemble models

In [ ]:
model_list = [## Boosting
              ('AB', AdaBoostRegressor()), 
              ('GBR', GradientBoostingRegressor()), 
              ## Bagging
              ('RF', RandomForestRegressor()), 
              ('ET', ExtraTreesRegressor())]

for name, model in model_list:
    model.fit(train_df, train_salary)
    validation_pred = model.predict(validation_features)
    mse = mean_squared_error(validation_salary, validation_pred)
    mae = mean_absolute_error(validation_salary, validation_pred)
    print("%s:\tMSE=%.3f\tMAE=%.3f" % (name, mse, mae))

# expect:
# AB:	MSE=555.790	MAE=19.869
# GBR:	MSE=364.269	MAE=15.487
# RF:	MSE=139.587	MAE=9.053
# ET:	MSE=107.677	MAE=6.012

In [ ]:
ensembles = []
# Boosting
ensembles.append(('scaledAB',
                  Pipeline([('Scaler', StandardScaler()),
                            ('AB', AdaBoostRegressor())])))
ensembles.append(('scaledGBR',
                  Pipeline([('Scaler', StandardScaler()),
                            ('GBR', GradientBoostingRegressor())])))
## Bagging
ensembles.append(('scaledRF',
                  Pipeline([('Scaler', StandardScaler()),
                            ('RF', RandomForestRegressor())])))
ensembles.append(('scaledET',
                  Pipeline([('Scaler', StandardScaler()),
                            ('ET', ExtraTreesRegressor())])))

for name, model in ensembles:
    model.fit(train_df, train_salary)
    validation_pred = model.predict(validation_features)
    mse = mean_squared_error(validation_salary, validation_pred)
    mae = mean_absolute_error(validation_salary, validation_pred)
    print("%s:\tMSE=%.3f\tMAE=%.3f" % (name, mse, mae))

# expect:
# scaledAB:	MSE=553.240	MAE=19.786
# scaledGBR:	MSE=364.269	MAE=15.487
# scaledRF:	MSE=139.061	MAE=9.046
# scaledET:	MSE=107.677	MAE=6.012

In [ ]:
train_df = data.copy()
model_list = [('LR', LinearRegression()), 
              ('LASSO', Lasso()), 
              ('EN', ElasticNet()), 
              ('CART', DecisionTreeRegressor()), 
              ('RF', RandomForestRegressor())]
validation_rows = 10000
for (model_name, model) in model_list:
    regression.score = sklearn.model_selection.cross_val_score(model, train.iloc[:,:-1], train.iloc[:,-1], scoring="neg_mean_squared_error")
    print(model_name, -1.0 * regression.score.mean())

#     model.fit(train_f, train_s)
#     valid_pred = model.predict(valid_f)
#     mse = mean_squared_error(valid_s, valid_pred)
#     mae = mean_absolute_error(valid_s, valid_pred)
#     print("model=%s,\tto_drop_companyId=%s, to_merge_degree_major=%s:\tMSE=%.3f\tMAE=%.3f" % 
#           (model_name, companyId, degree_major, mse, mae))
"""
model=LR,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=393.114	MAE=15.979
model=LASSO,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=394.213	MAE=16.006
model=EN,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=400.965	MAE=16.166
model=CART,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=686.648	MAE=20.350
model=RF,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=435.528	MAE=16.634
model=LR,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=393.119	MAE=15.980
model=LASSO,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=394.216	MAE=16.006
model=EN,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=400.967	MAE=16.166
model=CART,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=642.333	MAE=19.651
model=RF,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=438.890	MAE=16.668
model=LR,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=958.742	MAE=23.763
model=LASSO,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=948.846	MAE=23.606
model=EN,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=1111.485	MAE=25.481
model=CART,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=8083.295	MAE=82.648
model=RF,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=7966.888	MAE=82.662
model=LR,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=958.884	MAE=23.764
model=LASSO,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=948.890	MAE=23.607
model=EN,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=1111.605	MAE=25.482
model=CART,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=8006.305	MAE=82.538
model=RF,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=7948.391	MAE=82.554
"""

In [ ]:
train_df = data.copy()
test_list = [(True, False)]
model_list = [## Boosting
#               ('AB', AdaBoostRegressor()), 
              ('GBR', GradientBoostingRegressor(n_estimators=40, max_depth=7, loss='ls')), 
              ## Bagging
              ('RF', RandomForestRegressor(n_estimators=60, max_depth=15, min_samples_split=80, max_features=5)), ]
#               ('ET', ExtraTreesRegressor())]
validation_rows = 10000
for (companyId, degree_major) in test_list:
    for (model_name, model) in model_list:
        train = feature_engineer(to_drop_companyId=companyId, to_merge_degree_major=degree_major, train=data.copy())

        valid_f = train.iloc[:validation_rows, :-1]
        valid_s = train.iloc[:validation_rows, -1].values.astype(float)
        train_f = train.drop(index=train.index[:validation_rows])
        train_s = train_f.pop('salary')

        model.fit(train_f, train_s)
        valid_pred = model.predict(valid_f)
        mse = mean_squared_error(valid_s, valid_pred)
        mae = mean_absolute_error(valid_s, valid_pred)
        print("model=%s,\tto_drop_companyId=%s, to_merge_degree_major=%s:\tMSE=%.3f\tMAE=%.3f" % 
              (model_name, companyId, degree_major, mse, mae))

"""
Expect:
model=AB,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=545.306	MAE=19.586
model=GBR,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=366.419	MAE=15.532
model=RF,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=434.414	MAE=16.592
model=ET,	to_drop_companyId=False, to_merge_degree_major=False:	MSE=507.821	MAE=17.748
model=AB,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=555.479	MAE=19.755
model=GBR,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=366.419	MAE=15.532
model=RF,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=436.426	MAE=16.643
model=ET,	to_drop_companyId=True, to_merge_degree_major=False:	MSE=507.160	MAE=17.782
model=AB,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=10058.697	MAE=96.473
model=GBR,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=7936.804	MAE=83.056
model=RF,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=7964.971	MAE=82.636
model=ET,	to_drop_companyId=False, to_merge_degree_major=True:	MSE=8044.035	MAE=82.660
model=AB,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=10134.425	MAE=96.792
model=GBR,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=7936.804	MAE=83.056
model=RF,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=7949.010	MAE=82.555
model=ET,	to_drop_companyId=True, to_merge_degree_major=True:	MSE=7988.281	MAE=82.480
"""